In [ ]:
##Part 3: https://www.youtube.com/watch?v=4zE9HjPIqC4

#Here we parse the filingsummary.xml for each of the reports submitted
##https://github.com/areed1192/sigma_coding_youtube/blob/master/python/python-finance/sec-web-scraping/Web%20Scraping%20SEC%20-%2010K%20Landing%20Page%20-%20Single.ipynb
#import libraries
import requests #to make url request
import pandas as pd #transform data to more user-friendly version
from bs4 import BeautifulSoup #for parsing
import numpy as np
import time
##Beautiful Soup is powerful because our Python objects match the nested structure of the HTML document we are scraping.
##For deeply nested HTML documents, navigation could quickly become tedious. Luckily, Beautiful Soup comes with a search 
# function so we don't have to navigate to retrieve HTML elements.
##The find_all() method takes an HTML tag as a string argument and returns the list of elements 
# that match with the provided tag. For example, if we want all a tags in doc.html:



In [ ]:
##Importing merged Tyson data to extract information about all tickers of the companies we are interested in
#companies_data = pd.read_csv(r"C:/UCLAAnderson/10KfilingsTables/Merged_ofg_data_TysonAugust302022.csv", encoding='latin-1')
companies_data = pd.read_csv(r"C:/UCLAAnderson/10KfilingsTables/Data/AutomatedFinancialData/SP500ListCIKImportedfromWikipedia.csv", encoding='latin-1')
##formerly Feb242023_TruncatedAggregation.csv
companies_data.columns.tolist() ##colnames
## All top companies we are interested in
# comp_tickers = companies_data['CIK'].str.cat(sep=',').split(',')## to combine multiple rows of strings into one using pandas

# type(comp_tickers)
# xs = comp_tickers
# s = ','.join(xs)
# s

# companies_data.head()

In [ ]:
len(companies_data)
companies_data['Security'].tolist()

In [ ]:
## Getting CIK for all the TIKR symbols we need
# TIKR2CIK_url = 'https://www.sec.gov/files/company_tickers.json'
# content = requests.get(TIKR2CIK_url, headers = {'user-agent':'UCLA'})
# doc_content = content.json()
# doc_content

Rel_Companies = []
Rel_CIKCompanies = []
Rel_comp = {}
Companies = []
All22Comps=pd.DataFrame()
for idx, x in enumerate(list(doc_content.values())):
    if x['ticker'] in comp_tickers:
#         print(x)
        Companies.append(x)
        Rel_comp['CIK']= x['cik_str']
        
        Rel_CIKCompanies.append(Rel_comp['CIK'])
        print(Rel_CIKCompanies)#This is a list
        Rel_comp['TIKR']= x['ticker']
        Rel_comp['CompanyName']= x['title']
        
        All22Comps = pd.concat([All22Comps,pd.DataFrame(Companies)])

comp_names = companies_data['SEC.Filing.Name'].tolist()

lowCompNames = [x.lower() for x in comp_names]
lowCompNames

import re
CompNamesnoPuncLower=[]

for eachComp in list(range(0,len(lowCompNames),1)):
# initializing string
    test_str = lowCompNames[eachComp]

    # printing original string
    print("The original string is : " + test_str)

    # Removing punctuations in string
    # Using regex
    res = re.sub(r'[^\w\s]', '', test_str)

    # printing result
    print("The string after punctuation filter : " + res)
    
    CompNamesnoPuncLower.append(res)


CompNamesnoPuncLower

Rel_Companies = []
Rel_CIKCompanies = []
Rel_comp = {}
Companies = []
All22Comps = pd.DataFrame()

for idx, x in enumerate(list(doc_content.values())):
#     print(x['title'].lower())
    
    if x['title'].lower() in CompNamesnoPuncLower:
        print(x['title'].lower())
        Companies.append(x)
        Rel_comp['CIK'] = x['cik_str']
        Rel_CIKCompanies.append(Rel_comp['CIK'])
        print(Rel_CIKCompanies)  # This is a list
        Rel_comp['CompanyName'] = x['title']

        All22Comps = pd.concat([All22Comps, pd.DataFrame(Companies)])


All22Comps
All22Comps.drop_duplicates()

import requests
from bs4 import BeautifulSoup

def search_cik(company_name):
    base_url = "https://www.edgarcompany.sec.gov/servlet/CompanyDBSearch?page=main"
    search_url = f"{base_url}&companyname={company_name}"

    try:
        response = requests.get(search_url)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, "html.parser")

        cik_table = soup.find("table", class_="companyResultTable")
        rows = cik_table.find_all("tr")

        cik_results = []
        for row in rows[1:]:
            columns = row.find_all("td")
            cik = columns[0].text.strip()
            company = columns[1].text.strip()
            cik_results.append({"CIK": cik, "Company": company})

        return cik_results

    except requests.exceptions.RequestException as e:
        print(f"Error retrieving CIK for {company_name}: {e}")
        return None

# Example usage
company_name = "Apple Inc."
cik_results = search_cik(company_name)

if cik_results:
    print(f"CIK results for '{company_name}':")
    for result in cik_results:
        print(f"CIK: {result['CIK']}, Company: {result['Company']}")
else:
    print(f"No CIK results found for '{company_name}'")


for each in doc_content.values():
    print(each['title'])

companies_data['SEC.Filing.Name'].tolist()


import requests

def get_cik(company_names):
    base_url = "https://www.sec.gov/cgi-bin/browse-edgar"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    cik_dict = {}

    for company_name in company_names:
        params = {
            "action": "getcompany",
            "CIK": "",
            "output": "atom",
            "company": company_name
        }
        
        try:
            response = requests.get(base_url, headers={'user-agent':'UCLA'}, params=params)
            response.raise_for_status()
            cik = None
            
            # Extract CIK from the response
            for line in response.text.splitlines():
                if "<cik>" in line:
                    cik = line.split("<cik>")[1].split("</cik>")[0]
                    break
            
            if cik is not None:
                cik_dict[company_name] = cik
            else:
                cik_dict[company_name] = "CIK not found for the company."
            
        except requests.exceptions.RequestException as e:
            print(f"Error retrieving CIK for {company_name}: {e}")
            cik_dict[company_name] = None

    return cik_dict

# Example usage
company_names = ["Apple Inc.", "Microsoft Corporation", "Amazon.com Inc."]
cik_dict = get_cik(company_names)

# Print CIK values
for company_name, cik in cik_dict.items():
    print(f"CIK for {company_name}: {cik}")


import requests
import time

def get_cik(company_names):
    base_url = "https://www.sec.gov/cgi-bin/browse-edgar"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
    cik_dict = {}

    for company_name in company_names:
        params = {
            "action": "getcompany",
            "CIK": "",
            "output": "atom",
            "company": company_name
        }
        
        try:
            response = requests.get(base_url, headers=headers, params=params)
            response.raise_for_status()
            cik = None
            
            # Extract CIK from the response
            for line in response.text.splitlines():
                if "<cik>" in line:
                    cik = line.split("<cik>")[1].split("</cik>")[0]
                    break
            
            if cik is not None:
                cik_dict[company_name] = cik
            else:
                cik_dict[company_name] = "CIK not found for the company."
            
        except requests.exceptions.RequestException as e:
            print(f"Error retrieving CIK for {company_name}: {e}")
            cik_dict[company_name] = None
        
        time.sleep(1)  # Delay for 1 second between requests
        
    return cik_dict

# Example usage
company_names = companies_data['SEC.Filing.Name'].tolist()
cik_dict = get_cik(company_names)

# Print CIK values
for company_name, cik in cik_dict.items():
    print(f"CIK for {company_name}: {cik}")


## Getting CIK for all the TIKR symbols we need
TIKR2CIK_url = 'https://www.sec.gov/files/company_tickers.json'
content = requests.get(TIKR2CIK_url, headers = {'user-agent':'UCLA'})
doc_content = content.json()
doc_content

type(doc_content)
doc_content.items()
doc_content["0"]
doc_content.values()

SourceInfo=pd.DataFrame(doc_content.values(), columns=['cik_str', 'ticker','title'])
# SourceInfo.to_csv("C:/UCLAAnderson/10KfilingsTables/Data/AutomatedFinancialData/FinancialsOutput/May12_AllSourceCompanyMetadata.csv")

##Subsetted the entire nested dictionary based on the tickers that were extracted from Tyson's list

Rel_Companies = []
Rel_CIKCompanies = []
Rel_comp = {}
Companies = []
All500Comps=pd.DataFrame()
for idx, x in enumerate(list(doc_content.values())):
    if x['ticker'] in comp_tickers:
#         print(x)
        Companies.append(x)
        Rel_comp['CIK']= x['cik_str']
        
        Rel_CIKCompanies.append(Rel_comp['CIK'])
        print(Rel_CIKCompanies)#This is a list
        Rel_comp['TIKR']= x['ticker']
        Rel_comp['CompanyName']= x['title']
        
        All500Comps = pd.concat([All500Comps,pd.DataFrame(Companies)])
#         print(Rel_comp)
        



set_A=set(comp_tickers)
set_B = set(All500Comps['title'])
# set_A.difference(set_B) #for (A - B)
# len(set(All500Comps['title']))

type(comp_tickers)

diff=list(set(comp_tickers) - set(All500Comps['ticker']))

Missing22= companies_data[companies_data['Ticker.Symbol'].isin(diff)][['SEC.Filing.Name','Ticker.Symbol']]

Missing22.to_csv('C:/UCLAAnderson/10KfilingsTables/Data/AutomatedFinancialData/Missing22Tickers.csv')
# len(companies_data[companies_data['Ticker.Symbol'].isin(diff)][['SEC.Filing.Name','Ticker.Symbol']])

type(Rel_comp)
Rel_comp.keys()
pd.DataFrame(Companies)
All500Comps.drop_duplicates().to_csv("C:/UCLAAnderson/10KfilingsTables/Data/AutomatedFinancialData/FinancialsOutput/May16_All500CompanyMetadata.csv")

# print(Companies)
# len(Companies)
type(Companies)
pd.DataFrame(Companies).to_csv('C:/UCLAAnderson/10KfilingsTables/Data/May16_2023_CIKwithTIKRComps.csv')

## Export list of dictionaries to csv

import csv

to_csv = Companies

keys = to_csv[0].keys()

with open('C:/UCLAAnderson/10KfilingsTables/OurRelevantCompanySubset.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(to_csv)
    
    
    
import pandas as pd
df = pd.DataFrame(Rel_CIKCompanies)#, columns=["CIK"])
df.to_csv('C:/UCLAAnderson/10KfilingsTables/May16_2023_TikrCikOfRelCompanies.csv', index=False)
    

len(Rel_CIKCompanies)

SubComps_urls = []
master_list = []
for relevantCIK in Rel_CIKCompanies:
    # base URL for the SEC Edgar browser
    endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"

    ## defining a new variable parameter dictionary that will take the parameters' payload
    param_dict = {'action':'getcompany',
                  'CIK':relevantCIK,
                  'type':'10-K',
                  'dateb':'20220101',#####<<<<<<<<<<< before year 2023
                  'owner':'exclude',
                  'start':'',
                  'output':'',#html default
                  'count':5   
    }


    ## We will save the content that is sent back to us in a response variable:
    #  and then we will define the url as the endpoint and set out parameters to the param_dict
    response1 = requests.get(url = endpoint, headers = {'user-agent':'UCLA'}, params = param_dict)
    print('Request1 Successful')
    print(response1.url)
    SubComps_urls.append(response1.url)
    
    
    ##Parsing that response content and specify the parser - we know it is going to be html that comes back:
    soup = BeautifulSoup(response1.content,'html.parser')
    # find the document table with our data-- check using inspect how the table of all files is coded--it 
    #  has class = tablefile2 in the name

    doc_table = soup.find_all('table', class_='tableFile2')
    doc_table
    

len(doc_table)

## DIRECTLY USING CIKs I GATHERED - 20 OUT OF 500 COMPANIES IS JUST 4% OF ALL COMPANIES - AND THIS IS INCREDIBLY SUCCESSFUL AUTOMATION

In [ ]:
Rel_CIKCompanies=companies_data['CIK']

In [ ]:
Rel_CIKCompanies

In [ ]:
## Getting URLs for all relevant company subset using CIKs extracted earlier:
SubComps_Suburls = []
master_links = []
import time
start_time = time.time()
# main()
statements_url = [] ##empty list
statements_urlDF=[]

SecondResponses=[]
AllSoup2=[]

for relevantCIK in Rel_CIKCompanies:
    # base URL for the SEC Edgar browser
    endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"

    ## defining a new variable parameter dictionary that will take the parameters' payload
    param_dict = {'action':'getcompany',
                  'CIK':relevantCIK,
                  'type':'10-K',
                  'dateb':'20230101',#YYYMMDD<<<<<<<<<<< before year 2023
                  'owner':'exclude',
                  'start':'',
                  'output':'',#html default
                  'count':1}

##To try and get only the recent filings 4 quarters +1 to decide
                  
    ## We will save the content that is sent back to us in a response variable:
    #  and then we will define the url as the endpoint and set out parameters to the param_dict
    response1 = requests.get(url = endpoint, headers = {'user-agent':'UCLA'}, params = param_dict)
    print('Request1 Successful')
    print(response1.url)
    SubComps_Suburls.append(response1.url)
    
    
    ##Parsing that response content and specify the parser - we know it is going to be html that comes back:
    soup1 = BeautifulSoup(response1.content,'html.parser')
    # find the document table with our data-- check using inspect how the table of all files is coded--it 
    #  has class = tablefile2 in the name

    doc_table = soup1.find_all('table', class_='tableFile2')
    #define a base url that will be used for link building because you do not get the entire thing from href.

    base_url_sec = r"https://www.sec.gov"


    for row in doc_table[0].find_all('tr'):##didn't use placeholder 0 to get the recentmost filing because that's just table headers
        
        cols = row.find_all('td')

        # if there are no columns move onto the next row.
        if len(cols)!=0:#otherwise actually go onto grab the text
            filing_type = cols[0].text.strip()##strip the text for anything that is not useful
            filing_date = cols[3].text.strip()##strip the text for anything that is not useful
            filing_numb = cols[4].text.strip()##strip the text for anything that is not useful

            ##find the links
            #  I am going to look for an href that has the id = documentsbutton
            #The first parameter that has to be met is to see that href = true and the id should be 'documentsbutton'--from inspect

            filing_doc_href = cols[1].find('a',{'href':True, 'id':'documentsbutton'})
            filing_interactive_link = cols[1].find('a',{'href':True, 'id':'interactiveDataBtn'}) 
            ##In the proxy DEF 14A, filing_doc_href is not there, one has to click documents href and then go to the page with
            filing_num_href = cols[4].find('a')
            ##@18:37

        ## To actually get to the proxy statement, we need to click the filing_doc_href
            ##Grabbing the first href through the document button
            if filing_doc_href != None:
                filing_doc_link =base_url_sec+filing_doc_href['href']
            else:
                filing_doc_link = 'no link'

            ##Grabbing the last number href
            if filing_num_href != None:
                filing_num_link =base_url_sec+filing_num_href['href']
            else:
                filing_num_link = 'no link'
                
                
            ## This part for going to interactive data
            ##Grabbing the first href through the document button
            if filing_interactive_link != None:
                filing_table_link =base_url_sec+filing_interactive_link['href']
            else:
                filing_table_link = 'no link'


            ## Create and store parsed data in the dictionary
            # creating a dictionary and then setting a key where the key is equal to the value we parsed above

            file_dict = {}
            file_dict['file_type'] = filing_type
            file_dict['file_number'] = filing_numb
            file_dict['file_date'] = filing_date
            file_dict['links'] = {}
            file_dict['links']['documents'] = filing_doc_link
            file_dict['links']['Interactive Data'] = filing_table_link
            file_dict['links']['filing_number'] = filing_num_link
             ##To extract only the recentmost filing details i.e. 2021 for now:

            if '2021' in filing_date:#####<<<<<<<<<<< if it contains year 2022
            
                # let the user know it's working
                print('-'*100)        
                print("Filing Type: " + filing_type)
                print("Filing Date: " + filing_date)
                print("Filing Number: " + filing_numb)
                print("Document Link: " + filing_doc_link)
                print("Filing Number Link: " + filing_num_link)
                print("Interactive Data Link: " + filing_table_link)
                


                # append dictionary to master list
                
                file_dict1 = {}
                file_dict1['file_type'] = filing_type
                file_dict1['file_number'] = filing_numb
                file_dict1['file_date'] = filing_date
                file_dict1['links'] = {}
                file_dict1['links']['documents'] = filing_doc_link
                file_dict1['links']['Interactive Data'] = filing_table_link
                file_dict1['links']['filing_number'] = filing_num_link
                master_links.append(file_dict1)
                
                ##To avoid error because of Sweetgreen not having a filing as a recent company:
                if filing_table_link!='no link':
                    
                    response2 = requests.get(url = filing_table_link, headers = {'user-agent':'UCLA'})##going to the recent-most document DEF14A link

                    print('Request2 Successful')
                    print(response2.url)
                    print('Link to Tables')
                    print("/".join(filing_doc_link.split("/")[:8])+"/"+"index.json")##keeping the CIK and Accession number and removing everything after the 8th /

                    ##Parsing that response content and specify the parser - we know it is going to be html that comes back:
                    soup2 = BeautifulSoup(response2.content,'html.parser')
                    response2.content

                    SecondResponses.append(response2.content)
                    AllSoup2.append(soup2)

                    # soup2.prettify()

    #                 AllFilings = soup2.find_all('li', class_ = 'accordion')[0].find_all('a', class_="xbrlviewer")

                    AllFilings = soup2.find_all('li', class_ = 'accordion')[0].find_all('a', class_="xbrlviewer")
                    
                    Report_Name=[]

                    for pos in list(range(1,len(AllFilings),1)):
                        FilingName = AllFilings[pos].text.strip()
                        Report_Name.append(FilingName)

                    #Report_Name#[0:6]

                    ###### next


                    documents_url="/".join(filing_doc_link.split("/")[:8])+"/"+"index.json" ##Apple 10K

                                    ##Without the index.json, view the site and has a lot of files that are basically each just a table! 
                                    # For example, see https://www.sec.gov/Archives/edgar/data/1725516/000149315221032816/R2.htm
                                    #request the url and decode it.
                    headers = {'user-agent':'UCLA'}
                                    #headers = {'user-agent': 'sample text'}

                    content = requests.get(documents_url, headers=headers).json()
                                    ##Then loop through the json dictionary and go to the directory
                    for file in content['directory']['item']:

                                    #Grab the filing summary xml and create a new url leading to the file so we can download it.
                        if file['name']=='FilingSummary.xml':#Then we will create a path
                                        #https://note.nkmk.me/en/python-str-compare/
                                        #if re.search('.xml', file['name']):#trying to look for .xml instead
                            xml_summary = base_url_sec+content['directory']['name']+'/'+ file['name']
                                            ##name key returns the url except base_url
                            print('-'*100)
                            print('File name: ' + file['name'])
                            print('File Path: ' + xml_summary)
                                            #@9:36

                        content
                                # define a new base url that represents the filing folder. This will come in handy when we need to download the reports.
                    base_url = xml_summary.replace('FilingSummary.xml','')
                            #base_url # coming to the directory list by removing last part of the link to land at the place where each report is placed together

                            # request and parse the content
                    content = requests.get(xml_summary, headers = headers).content
                            #content ## the dot content will list out the nestedness b'<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<FilingSummary>\n  <Version>3.19.1</Version>\n  <ProcessingTime/>\n  <ReportFormat>html</ReportFormat>\n  <ContextCount>224</ContextCount>\n  <ElementCount>324</ElementCount>\n  <EntityCount>1</EntityCount>\n  <FootnotesReported>false</FootnotesReported>\n  <SegmentCount>73</SegmentCount>\n  <ScenarioCount>0</ScenarioCount>\n  <TuplesReported>false</TuplesReported>\n  <UnitCount>7</UnitCount>\n  <MyReports>\n    <Report instance="mtii-20181231.xml">\n      <IsDefault>false</IsDefault>\n      <HasEmbeddedReports>false</HasEmbeddedReports>\n      <HtmlFileName>R1.htm</HtmlFileName>\n      <LongName>0001000 - Document - Document and Entity Information</LongName>\n      <ReportType>Sheet</ReportType>\n      <Role>http://www.monitronics.com/role/DocumentAndEntityInformation</Role>\n      <ShortName>Document and Entity Information</ShortName>\n      <MenuCategory>Cover</MenuCategory>\n      <Position>1</Position>\n    </Report>\n    <Report instance="mtii-20181231.xml">\n      <IsDefault>false</IsDefault>\n      <HasEmbeddedReports>false</HasEmbeddedReports>\n      <HtmlFileName>R2.htm</HtmlFileName>\n      <LongName>1001000 - Statement - Consolidated Balance Sheets</LongName>\n      <ReportType>Sheet</ReportType>\n      <Role>http://www.monitronics.com/role/ConsolidatedBalanceSheets</Role>\n      <ShortName>Consolidated Balance Sheets</ShortName>\n      <MenuCategory>Statements</MenuCategory>\n      <Position>2</Position>\n    </Report>\n    <Report instance="mtii-20181231.xml">\n      <IsDefault>false</IsDefault>\n      <HasEmbeddedReports>false</HasEmbeddedReports>\n      <HtmlFileName>R3.htm</HtmlFileName>\n      <LongName>1001501 - Statement - Consolidated Balance Sheets (Parenthetical)</LongName>\n      <ReportType>Sheet</ReportType>\n      <Role>http://www.monitronics.com/role/ConsolidatedBalanceSheetsParenthetical</Role>\n      <ShortName>Consolidated Balance Sheets (Parenthetical)</ShortName>\n      <MenuCategory>Statements</MenuCategory>\n      <Position>3</Position>\n    </Report>\n    <Report instance="mtii-20181231.xml">\n      <IsDefault>false</IsDefault>\n      <HasEmbeddedReports>false</HasEmbeddedReports>\n   

                    soup = BeautifulSoup(content, 'lxml')
                            #soup ##Beautifying the nestedness for easier reading so that \n is a different line so its closer to how it looks on https://www.sec.gov/Archives/edgar/data/1265107/000126510719000004/FilingSummary.xml

                            ##Then, find the 'MyReports' tag because it contains all the individual reports submitted
                    reports = soup.find('myreports')
                            # reports: all filed reports under the tag my reports

                                    ## I want a list to store all the individual components of the report, so creating the master list:
                    master_reports = []

                        
                    for report in reports.find_all('report')[:-1]:

                                # Let's create an empty dictionary - curly brackets
                        report_dict = {}
                                ##then assign new keys to it and each value assigned to the keys is a different portion of the report tag
                        report_dict['name_short'] = report.shortname.text ##get only the text from the tag ShortName
                        report_dict['name_long'] = report.longname.text ##get only the text from the tag ShortName
                        #report_dict['position'] = report.position.text ##get only the text from the tag ShortName
                        report_dict['category'] = report.menucategory.text ##get only the text from the tag ShortName
                        report_dict['url'] = base_url + report.htmlfilename.text ##get only the text from the tag ShortName

                                ## We need to append this to the master report we created outside the for loop
                        master_reports.append(report_dict)

                         
                    for report in master_reports:

                                #Define the statements we want to look for
                        item1 = r"Consolidated Balance Sheets" ##r' means raw string
                        item2 = r"Consolidated Statements of Operations and Comprehensive Income (Loss)"
                        item3 = r"Consolidated Statements of Cash Flows"
                        item4 = r"Consolidated Statements of Stockholder's (Deficit) Equity"
                        item5 = r"Property and Equipment"##Capital Expenditure
                        item6 = r"Consolidated Statements of Income"
                        item7 = r"Income Taxes"
                        item8 = r"Property and Equipment"

                           

                                #storing them in a list
                        titles = [item1.lower(), item2.lower(), item3.lower(), item4.lower(), item5.lower(), item6.lower(),
                                  item7.lower(),item8.lower()] ## + means concatenate and comma means or

                        keywords = ["cover","balance sheets", "of operations","of earning",
                                    "cash flow","income tax","property and equipment","statement","statements"]

                        test_string = report['name_short'].lower()

                        # initializing test list
                        test_list = keywords
                        

                        flag=0
                        #for i in test_string:
                        for j in test_list:
                            if j in test_string and "quarterly" not in test_string:
                                #and "(" not in test_string is leading to removal of statements that have (loss) in the title
                                        #if all(x not in i for x in mylist):
                                flag=1
                                break#https://www.geeksforgeeks.org/python-test-if-string-contains-element-from-list/
                        if flag==1:
                                    #print("String contains the list element")
                                                    ## Print some info and store it in the statements_url
                            print('-'*100)
                            print(report['name_short'])
                            print(report['url'])

    #                         statements_url.append(report['url'])
    #                         statements_url.append(,ignore_index=True)
                            url_df = pd.DataFrame({'cik_str':[relevantCIK],'URL':[report['url']], 'TableName':[report['name_short']],'LinkToAllTabs':response2.url})
                            statements_urlDF.append(url_df)
                            statements_url.append(report['url'])

                            #     else:
                            #         print("String does not contain the list element")


                        #statements_url

time_taken = (time.time() - start_time)

In [ ]:
# type(pd.concat(statements_url))
type(statements_url[0])
statements_url
pd.DataFrame (statements_url, columns = ['URL'])

In [ ]:
## Exporting list of dataframes to csv for later use/import

pd.concat(statements_urlDF).to_csv('C:/UCLAAnderson/10KfilingsTables/Data/AutomatedFinancialData/July13_SP503WikiCompanies_statements_urlDF.csv')
pd.DataFrame (statements_url, columns = ['URL']).to_csv('C:/UCLAAnderson/10KfilingsTables/Data/AutomatedFinancialData/July13_SP503WikiCompanies_statements_url.csv')

In [ ]:
# ##Maybe include a column for the link to request 2 so that NA's in tables can be cross-checked

# statements_list2df = pd.concat(statements_urlDF)##to convert list of pandas dataframes to a dataframe
# ##merge with the Companies dataframe to get company names and TIKR
# CompanyInfo = pd.DataFrame(Companies, columns = ['cik_str','ticker','title','ForCrossCheckingNAtables'])

# ##merge the two dataframes by CIK:
# Company10KfilingsURL = pd.merge(CompanyInfo, statements_list2df, on ='cik_str')
# ##Export this metadata

# Company10KfilingsURL.to_csv('C:/UCLAAnderson/10KfilingsTables/July6_22MissingCompanies_before2021_ShortenedCompanyMetadataAnd10KTableURLs.csv', index=False)
    

In [ ]:
'110825' in statements_url
type(statements_url)
matching = [s for s in statements_url if "1108524" in s]
matching
# len(statements_data)
# iterate through list of elements
# for i in statements_url:
   
#     # check for type is str
#     if(type(i) is str):
       
#         # display index
#         print(statements_url.index(i))
###finding the location/index where salesforce links are stored in this list ##https://stackoverflow.com/questions/2170900/get-first-list-index-containing-sub-string
index = [idx for idx, s in enumerate(statements_url) if '1108524' in s][0]
index

statements_url[7897]

# TRIAL FOR GETTING $MILLIONS UNITS

In [ ]:
##TRYING TO CLEAN DATA ALONGSIDE

import re
##Finally the correct code to get three dataframes separately for all companies
##Remaining pseudocoding for headers in income tax as they vary from one date, two dates to three dates


##useful functions:

## Let's assume we want all the statements in a single dataset.
# del each
start_AllTables=time.time()
def flatten(l):
    return [item for sublist in l for item in sublist]
# list(range(len(statements_data)))

def concat(*args):
    return pd.concat([x for x in args if not x.empty])

##Empty lists

statements_data = [] ## empty master list with all the statements
AllTablesOfACompany=[]
Allstatements_report_soup=[]
# ##Start with empty dataframes:
# Company10KData=pd.DataFrame(columns = [0, 1, 2, 3, 'Company', 'cik_str'])
# OperationsStatement=pd.DataFrame(columns = [0, 1, 2, 3, 'Company', 'cik_str'])
# TotalUStax=pd.DataFrame(columns = [0, 1, 2, 3, 'Company', 'cik_str'])
# ResearchDev=pd.DataFrame(columns = [0, 1, 2, 3, 'Company', 'cik_str'])


# Loop through each statement url:
for statement in statements_url:
    
    #define a dictionary that will store the different parts of the statement
    ##After inspecting (after right clicking the table in https://www.sec.gov/Archives/edgar/data/1265107/000126510719000004/R2.htm)
    ## I find that I expect to have different sections of my financial statement
    ## so I just store each section and the corresponding component of the actual statement data dictionary
    
    statement_data = {} # empty dictionary
    statement_data['headers']=[] #empty list
    statement_data['sections'] = []
    statement_data['data'] = [] #actual numbers
    
    headers = {'user-agent':'UCLA'}
    #Request the statement file content
    content = requests.get(statement, headers = headers).content #get content of the requested url
    
    # Create a new variable for parsing
    report_soup = BeautifulSoup(content, 'html') # In this example the parser is HTML
    Allstatements_report_soup.append(report_soup)
    
    
    for index, row in enumerate(report_soup.table.find_all('tr')): 
    #----> tr is how all the rows are stated inside table when inspected
    ##Again look in the drop-down of tr row you see 'td' which is basically individual element or each column 
    # We will basically find all the elements of any given row and store it in anew variable called columns
    
        cols = row.find_all('td')
    
    
    # So,creating a logic statement with this observation
    # if its a regular row snd not a section or a table header
    
        if(len(row.find_all('th'))==0 and len(row.find_all('strong'))==0):
            # next we are going to declare a new variable and then do list comprehension to parse each individual element
            # for element in column as defined above cols, we will say element.text.strip 
            # for each element 
            
            reg_row = [ele.text.strip() for ele in cols] #- this will be stored in a list
            statement_data['data'].append(reg_row)
            
        ## if a section row:
        elif(len(row.find_all('th'))==0 and len(row.find_all('strong'))!=0):
            
            sec_row = cols[0].text.strip() ### all I really want in a section row is the name of the section
            statement_data['sections'].append(sec_row) ## Appending it to the sections part of the master statement
            
            ## if a header row
        elif(len(row.find_all('th'))!=0): #because some table headers may or may not have a strong tag
            hed_row = [ele.text.strip() for ele in row.find_all('th')] #- instead of looking across column, now looking across rows for the category names
            statement_data['headers'].append(hed_row)
            
        else:
            print('We encountered an error.')

            
            ##We finally need to append this to the master list
    statements_data.append(statement_data)

In [ ]:
statements_data_DF=pd.DataFrame(statements_data)

statements_data_DF.to_csv('C:/UCLAAnderson/10KfilingsTables/Data/AutomatedFinancialData/ForReadingIn/July13_SP503WikiCompanies_Statements_data_DF.csv')

In [ ]:
# statements_dataOut = pd.read_csv('C:/UCLAAnderson/10KfilingsTables/Data/AutomatedFinancialData/ForReadingIn/April13_Statements_data_DF.csv')

In [ ]:
unit_headers=[]
for each1 in list(range(0,len(statements_data))):
    print(statements_data[each1]['headers'][0][0])
    print(each1)
    if '$ in' in statements_data[each1]['headers'][0][0].lower():
        unit_headers.append(statements_data[each1]['headers'][0][0].lower().split("$ in",1)[1])

In [ ]:
statements_data[40]

In [ ]:
def listNum(myList, x):
    return [item for item in myList if type(item) is str and len(item) == x]

In [ ]:
import csv
with open('July13_SP503WikiUniqueUnitsUsed.csv', 'w',encoding="utf-8") as f:
    writer = csv.writer(f)
    for url in set(unit_headers):
        writer.writerow([url])

## Saving the statements_data list of dictionaries because it takes a long time to run

## Importing statements_data saved earlier for 2021 to avoid spending hours in running the previous code

In [ ]:
len(statements_data)
statements_data
statements_urlDF

In [ ]:
# worklist = list(range(0,len(statements_data),1))
# batchsize = 500

# for each in range(0, len(worklist), batchsize):
#     batch = worklist[each:each+batchsize]
# #     print(batch)
    
#     for each in batch:
#         print(each)

In [ ]:
len(statements_data)

### Original with single loop####

In [ ]:
# conda install -c conda-forge nbstripout

In [ ]:
##Start with empty dataframes:

DFOperationsStatement1=pd.DataFrame()
OuterDFOperationsStatement=pd.DataFrame()
DF_RandD=pd.DataFrame()
DFTaxStats=pd.DataFrame()

##and with empty lists:
List_RandD=[]
ListTaxStats=[]
Company10KData=[]
CompanyFinancials=[]
OperationsStatement=[]
TotalUStax=[]
ResearchDev=[]
RandD=[]
TaxnOperationsStatement=[]
TaxOperationsRnD_Statement=[]
# Ops_sel=[]
Ops_sel=pd.DataFrame()
NetRevenue=[]
NetSales=[]
OperatingCost=[]
EmployeeWages=[]
df1B=pd.DataFrame()
df_out1B=pd.DataFrame()
df_Revenue=pd.DataFrame()

df_out1B=pd.DataFrame()

##Iterating over a list of dictionaries
# for each in list(range(0,len(statements_data),1)):#     for key in statements_data[index]:
# worklist = list(range(0,len(statements_data),1))
# batchsize = 500

# for Outereach in range(0, len(worklist), batchsize):
#     batch = worklist[Outereach:Outereach+batchsize] # the result might be shorter than batchsize at the end
    # do stuff with batch
    ## Grabbing the proper components
    ## headers and data are lists of lists so making it a list instead
# for each in list(range(0,3,1)):
# for each in list(range(0,len(statements_data),1)):#batch:
# for each in list(range(0,round(len(statements_data)/2),1)):#batch:
for each in list(range(round(len(statements_data)/2),len(statements_data),1)):#batch:

    income_headers = statements_data[each]['headers']##for taking the dates stored as the second list in headers of 
                                                     ##Consolidated Statements of Operations and Comprehensive Income (Loss) - USD ($) $ in Thousands

        ##The balance sheet has only one header but if you go to the income statement it has two headers 
        ##we take [1] because we were interested in the dates header which was the second element

    income_data = statements_data[each]['data']
    # income_data ##here we have each row and the corresponding value for each particular financial statement- different sections


    ## Put the data in a dataframe
    income_df = pd.DataFrame(income_data)

    ##For displaying

#     print('-'*100)
#     print('Before Reindexing')
#     print('-'*100)
#     display(income_df.head(7)) ## here we see the problem
    ## first problem is that index is actually the Net Revenue, cost of services etc....

    ## So, define the index column, rename it, and we need to make sure to drop the old column once we reindex.

    income_df.index = income_df[0]##column 0 is first column-- something like row.names in R



    ## Sometimes before removing the first column to row.names instead of replacing you can also assign it a new intuitive name

    income_df.index.name = 'Category'
    ##now dropping the first column which is now a rowname or index:
    income_df = income_df.drop(0, axis = 1) ##drop method takes the column number to be dropped and along which axis 
                                            ##- 1 means along column axis


    ## Now, we see problem 2, which is the dollar signs -  which tells me that right now it is a string
    ## this being a string is a problem because I can't do any calculations on a string
    ## Additionally, the blank values should be represented as a non-value (Problem 3)
    ## 4th problem: negative values in losses should be true negatives and not have $

    ## So, running some replace operations on this dataframe

    ## Get rid of the '$', '(', ')', and convert the '' to NaNs. @44:34
    #dealing with replacing positive values
    #then, dealing with negative values such that if you see closing bracket replace with minus sign
    # finally, if you see empty cells, replace with NaNs non values
    income_df = income_df.replace('[\$,)]','', regex = True)\
    .replace('[(]','-', regex = True)\
    .replace('',np.nan, regex = True)


    CompNameCIK=str()
    NetRevenue=[]
    NetSales=[]
    OperatingCost=[]
    EmployeeWages=[]
#         OperatingCost=pd.DataFrame()
#         Ops_sel=pd.DataFrame()
#         ops=pd.DataFrame()
    ##Now that we aren't relying on the cover page to get the name of 

    soupy=BeautifulSoup(requests.get(statements_urlDF[each]['LinkToAllTabs'][0], headers = headers).content,'html.parser')

    node = soupy.find('span', class_='companyName')
    if node is not None:
        ementa = node.text
        CompNameCIK=soupy.find('span', class_='companyName').text.strip().split('CIK:')
        CompanyName = CompNameCIK[0]
        CIK = CompNameCIK[1]
#     else:
#         ementa = None




    if 'operation' in statements_data[each]['headers'][0][0].lower() or 'income' in statements_data[each]['headers'][0][0].lower() or 'earning' in statements_data[each]['headers'][0][0].lower():
#         or 'cash flow' in statements_data[each]['headers'][0][0].lower():
#     or 'consolidated' in statements_data[each]['headers'][0][0].lower() or 'statement' in statements_data[each]['headers'][0][0].lower() or 'sheet' in statements_data[each]['headers'][0][0].lower():



####################################################################################################################   



        NetRevenue = pd.DataFrame(statements_data[each]['data']).loc[pd.DataFrame(statements_data[each]['data']).iloc[:,0].apply(lambda x: x.lower()).str.contains('|'.join(['revenue','revenues',
        'sale','sales','net loss','net (loss)',
        'net interest income','total operating','interest income','net interest',
#         'net cash','earning','net other',
#         'asset','liabilit',
#         'general and administrative','general & administrative',#selling, 
        'research and development','research & development', 'R&D','R & D',
#         'effective tax rate','effective rate'
])),:]

#######Using "in" instead of str.contains:
#         stat_df=pd.DataFrame(statements_data[each]['data'])
#         NetRevenue = stat_df[stat_df[0].apply(lambda x: any(term in x.lower() for term in ['revenues', 'revenue','sale','sales',
#                                                                                  'net loss','net (loss)','net interest income',
#                                                                                  'total operating','interest income','net interest',
#                                                                                            'research and development','research & development', 'R&D','R & D']))]

#         df1=NetRevenue.replace("\$",'',regex=True)

############## ********** Make sure to add backward slash to avoid replacing the values 1 and 2 occuring in revenue values
        df = NetRevenue[NetRevenue.columns].replace({'\(':'-','\)':'','â€‹':'','%':'','\$':'',' ':'',',':'','\[1]':'','\[2]':''}, regex = True)
    # using apply method
        cols = df.columns.drop(df.columns[0])
        df[cols]=df[cols].apply(lambda x: pd.to_numeric(x, errors='coerce'))####Converting non metric columns to numeric to be able to multiply later with


        if len(df)!=0:
            df_Revenue=df

            if '$ in millions' in statements_data[each]['headers'][0][0].lower():# and '$' in NetRevenue:
               #and NetRevenue>1000  
            ##Cleaning the data before building it:
                s =df_Revenue[df_Revenue.select_dtypes(include=['number']).columns]*1000000
                df[s.columns] = s
            elif '$ in thousands' in statements_data[each]['headers'][0][0].lower():# and '$' in NetRevenue:
                s =df_Revenue[df_Revenue.select_dtypes(include=['number']).columns]*1000
                df[s.columns] = s
            elif '$ in billions' in statements_data[each]['headers'][0][0].lower():# and '$' in NetRevenue:
                s =df_Revenue[df_Revenue.select_dtypes(include=['number']).columns]*1000000000
                df[s.columns] = s
                ## below is an exception
    #         elif '$ in millions, $ in billions' in statements_data[each]['headers'][0][0].lower():# and '$' in NetRevenue:
    #             # and NetRevenue<1000 
    #             df_Revenue =df_Revenue[df_Revenue.select_dtypes(include=['number']).columns]*1000000000
            else:
                s=df_Revenue[df_Revenue.select_dtypes(include=['number']).columns]*1
                df[s.columns] = s

            df1B_duped = pd.concat([df[df.columns[0]], df_Revenue], axis=1)


            ####Dedupe column names right here to change colnames ############


            df1B_dedupe = df1B_duped.loc[:,~df1B_duped.columns.duplicated()].copy()

            ##for later
            df1B=df1B_dedupe.dropna(axis=0)
    ## Below is to avoid reindex error due to repeated columns like deferred tax revenue being reported in two columns acc to: https://stackoverflow.com/questions/14984119/python-pandas-remove-duplicate-columns
    #         df1B = pd.concat([df[df.columns[0]], df_Revenue], axis=1).loc[:,~(pd.concat([df[df.columns[0]], df_Revenue], axis=1)).columns.duplicated()].copy()

    #             df1B = df_Rev[df_Rev.columns].replace({'\(':'-','\)':'','â€‹':'','%':'','\$':'',' ':'',',':''}, regex = True)
        #         '\$':''<<-removed this condition because if there is $ in front of the value, the headers direct the units thousand or million

                #         if '$'in df1B:
                    #multiply bymillion or thousand
#             df_out1B=pd.DataFrame()
            ##2. Replace NanNs with next column's non-NaN value
            if df1B.isnull().values.any():
                df_out1B = df1B.fillna(method='ffill',axis=0).dropna(axis=1)
    #                 df_out1B = df1B.apply(lambda x: pd.Series(x.dropna().to_numpy()), axis=1)
    #                 df_out1B = df_out1B.set_axis(df1B.columns[:df_out1B.shape[1]], axis=1).reindex(df1B.columns, axis=1).dropna(axis=1)
            else:
                df_out1B = df1B.dropna(axis=1)
            print(df_out1B)

            ##3. replacing empty whitespace with NaN---do we need to do this again?
            cleaned_NetRevenue=df_out1B.replace(r'^\s*$', np.nan, regex=True).dropna(axis=1, how='all')


            if len(CompNameCIK)!=0:
                cleaned_NetRevenue['Company']= CompanyName
                cleaned_NetRevenue['cik_str']= CIK
            else:
                cleaned_NetRevenue['Company']= 'NotFound'
                cleaned_NetRevenue['cik_str']= 'NotFound'
            print(cleaned_NetRevenue) 

    #######################To avoid the initial columns being 0,1,2,3 followed with opsheader dates##############


    ##################################################################################
        ##To avoid clumping all dates into a single list:
            if len(statements_data[each]['headers'])>1:
                        dates_init = statements_data[each]['headers'][1]
            else:
                dates_init = statements_data[each]['headers'][0]


            dateslist_initial=[]
            for onedate in dates_init:
            #     print(re.sub(r'(,?!(([^"]*"){2})*[^"]*$)',':', str(onedate)))
                mystring1 = re.sub(r'(,?!(([^"]*"){2})*[^"]*$)',':', str(onedate))
                print(re.sub(",", "/", mystring1))
                datestr1a=re.sub(",", "/", mystring1)## replace comma with / before the year
                if len(datestr1a.split("/"))>1:
                    datestr1=datestr1a.split("/")[0].split(" ")[0]+" "+datestr1a.split("/")[1]
                    ##To drop the exact date, splitting at \ for year and then splitting at space to keep Month
                else:
                    datestr1=datestr1a.split("/")[0].split(" ")[0]+" "+datestr1a.split("/")[0]
                dateslist_initial.append(datestr1)

            print(dateslist_initial)
            singledatesstring1 = ','.join(dateslist_initial)
            datesList1_init = re.sub(",", ",", singledatesstring1).split(',')

            CleanRevHeaders = ['Metric']


            for each1 in datesList1_init:
                print('-'.join(each1.split(',')))
                CleanRevHeaders.append(''.join(each1.split(',')))


            import itertools
            AllCleanRevHeaders = list(itertools.chain(CleanRevHeaders, ['Company'], ['cik_str']))

            if len(cleaned_NetRevenue.columns.tolist())==len(AllCleanRevHeaders):
                cleaned_NetRevenue.columns=AllCleanRevHeaders

            print(cleaned_NetRevenue)
        ##################################################################################

    ####################################################################################################################


    ##Some tables might not be there like R&D for some companies:
            if (len(cleaned_NetRevenue))!=0:#+len(cleaned_NetSales)+len(cleaned_OperatingCost)+len(cleaned_EmployeeWages)+len(cleaned_RandDOpsAmgen))!=0:
                #ops = concat(*(cleaned_NetRevenue))#, cleaned_NetSales,cleaned_OperatingCost, cleaned_EmployeeWages,cleaned_RandDOpsAmgen))
                ops=cleaned_NetRevenue
                if len(ops)!=0:

                    editedOps = pd.DataFrame(ops[ops.columns[ops.notnull().all()]].iloc[0].dropna())
                    editedOps.dropna(inplace=True)
                    editedOps
                    # remove empty column that just takes up space and shifts the 3 years' data one cell further
                    CleanOps = editedOps.loc[:, editedOps.notna().any()]
                    Operations_edit = pd.DataFrame(CleanOps.replace(r'^\s*$%', np.nan, regex=True).dropna())
                    #TotalUStax_edit.where(~TotalUStax_edit.apply(lambda x: x.astype(str).str.contains('|'.join(['%','$'])))).dropna().T




        #             Ops_sel = [ops]
                    Ops_sel = Operations_edit.T
        #             dates = statements_data[each]['headers'][0]
                    if len(statements_data[each]['headers'])>1:
                        dates = statements_data[each]['headers'][1]
                    else:
                        dates = statements_data[each]['headers'][0]

    #                     Ops_sel.columns = ['Metric',dates,'Company','cik_str']##<<<<to see if date can be registered


        ##################################################################################
        ##To avoid clumping all dates into a single list:

                    dateslist1=[]
                    for onedate in dates:
                    #     print(re.sub(r'(,?!(([^"]*"){2})*[^"]*$)',':', str(onedate)))
                        mystring1 = re.sub(r'(,?!(([^"]*"){2})*[^"]*$)',':', str(onedate))
                        print(re.sub(",", "/", mystring1))
                        datestr1a=re.sub(",", "/", mystring1)## replace comma with / before the year
                        if len(datestr1a.split("/"))>1:
                            datestr1=datestr1a.split("/")[0].split(" ")[0]+" "+datestr1a.split("/")[1]
                            ##To drop the exact date, splitting at \ for year and then splitting at space to keep Month
                        else:
                            datestr1=datestr1a.split("/")[0].split(" ")[0]+" "+datestr1a.split("/")[0]
                        dateslist1.append(datestr1)

                    print(dateslist1)
                    singledatesstring1 = ','.join(dateslist1)
                    datesList1 = re.sub(",", ",", singledatesstring1).split(',')

                    OpsHeaders = ['Metric']


                    for each1 in datesList1:
                        print('-'.join(each1.split(',')))
                        OpsHeaders.append(''.join(each1.split(',')))


                    import itertools
                    AllOpsHeaders = list(itertools.chain(OpsHeaders, ['Company'], ['cik_str']))

                    if len(Ops_sel.columns.tolist())==len(AllOpsHeaders):
                        Ops_sel.columns=AllOpsHeaders
        ##################################################################################
        #         else:
        #             Ops_sel = pd.DataFrame(['NIL'])

        ##### Checking for duplicated columns like deferred revenues that create problems in rbind:

            Ops_sel_dedupe = Ops_sel.loc[:,~Ops_sel.columns.duplicated()].copy()
            OperationsStatement.append(Ops_sel_dedupe)
            print(OperationsStatement)


            ##Also creating a dataframe instead of a list of dataframes for easier understanding of column values
            DFOperationsStatement1 = pd.concat([DFOperationsStatement1, Ops_sel_dedupe])

            # Define the filename for the current batch
    #             filename = f'batch_{each+1}_{Outereach+1}.csv'
    #             print(filename)
    #             batch_df=DFOperationsStatement
            # Export the current batch to a CSV file
    #             batch_df.to_csv(filename, index=False)
    #             batch_df.to_csv(DFOperationsStatement, index=False)
else:
    print("*"*100)
    print('None of the listed tables')
    print("*"*100)



    CompanyFinancials = [OperationsStatement]#,TotalUStax,RandD]
AllTablesOfACompany.append(CompanyFinancials)
    

# OuterDFOperationsStatement= pd.concat([OuterDFOperationsStatement, DFOperationsStatement])


end_AllTables = time.time()

timeTaken = end_AllTables-start_AllTables

DFOperationsStatement1.to_csv('C:/UCLAAnderson/10KfilingsTables/Data/AutomatedFinancialData/July17_SP223to503WikiCompanies_Part4b_2023_withMetricsnDate2021nUnits_RevenueRnD_SP500.csv', index=False)


In [ ]:
statements_data[7898]

In [ ]:
DFOperationsStatement1

In [ ]:
# round(len(statements_data)/2)
# len(df)
len(set(DFOperationsStatement1['Company']))

In [ ]:
##Start with empty dataframes:

DFOperationsStatement2=pd.DataFrame()
OuterDFOperationsStatement=pd.DataFrame()
DF_RandD=pd.DataFrame()
DFTaxStats=pd.DataFrame()

##and with empty lists:
List_RandD=[]
ListTaxStats=[]
Company10KData=[]
CompanyFinancials=[]
OperationsStatement=[]
TotalUStax=[]
ResearchDev=[]
RandD=[]
TaxnOperationsStatement=[]
TaxOperationsRnD_Statement=[]
Ops_sel=[]
NetRevenue=[]
NetSales=[]
OperatingCost=[]
EmployeeWages=[]
df1B=pd.DataFrame()
df_out1B=pd.DataFrame()
df_Revenue=pd.DataFrame()

##Iterating over a list of dictionaries
# for each in list(range(0,len(statements_data),1)):#     for key in statements_data[index]:
# worklist = list(range(0,len(statements_data),1))
# batchsize = 500

# for Outereach in range(0, len(worklist), batchsize):
#     batch = worklist[Outereach:Outereach+batchsize] # the result might be shorter than batchsize at the end
    # do stuff with batch
    ## Grabbing the proper components
    ## headers and data are lists of lists so making it a list instead
for each in list(range(((round(len(statements_data)/2))+1),len(statements_data))):#batch:
    income_headers = statements_data[each]['headers']##for taking the dates stored as the second list in headers of 
                                                     ##Consolidated Statements of Operations and Comprehensive Income (Loss) - USD ($) $ in Thousands

        ##The balance sheet has only one header but if you go to the income statement it has two headers 
        ##we take [1] because we were interested in the dates header which was the second element

    income_data = statements_data[each]['data']
    # income_data ##here we have each row and the corresponding value for each particular financial statement- different sections


    ## Put the data in a dataframe
    income_df = pd.DataFrame(income_data)

    ##For displaying

#     print('-'*100)
#     print('Before Reindexing')
#     print('-'*100)
#     display(income_df.head(7)) ## here we see the problem
    ## first problem is that index is actually the Net Revenue, cost of services etc....

    ## So, define the index column, rename it, and we need to make sure to drop the old column once we reindex.

    income_df.index = income_df[0]##column 0 is first column-- something like row.names in R



    ## Sometimes before removing the first column to row.names instead of replacing you can also assign it a new intuitive name

    income_df.index.name = 'Category'
    ##now dropping the first column which is now a rowname or index:
    income_df = income_df.drop(0, axis = 1) ##drop method takes the column number to be dropped and along which axis 
                                            ##- 1 means along column axis


    ## Now, we see problem 2, which is the dollar signs -  which tells me that right now it is a string
    ## this being a string is a problem because I can't do any calculations on a string
    ## Additionally, the blank values should be represented as a non-value (Problem 3)
    ## 4th problem: negative values in losses should be true negatives and not have $

    ## So, running some replace operations on this dataframe

    ## Get rid of the '$', '(', ')', and convert the '' to NaNs. @44:34
    #dealing with replacing positive values
    #then, dealing with negative values such that if you see closing bracket replace with minus sign
    # finally, if you see empty cells, replace with NaNs non values
    income_df = income_df.replace('[\$,)]','', regex = True)\
    .replace('[(]','-', regex = True)\
    .replace('',np.nan, regex = True)


    CompNameCIK=str()
    NetRevenue=[]
    NetSales=[]
    OperatingCost=[]
    EmployeeWages=[]
#         OperatingCost=pd.DataFrame()
#         Ops_sel=pd.DataFrame()
#         ops=pd.DataFrame()
    ##Now that we aren't relying on the cover page to get the name of 

    soupy=BeautifulSoup(requests.get(statements_urlDF[each]['LinkToAllTabs'][0], headers = headers).content,'html.parser')

    node = soupy.find('span', class_='companyName')
    if node is not None:
        ementa = node.text
        CompNameCIK=soupy.find('span', class_='companyName').text.strip().split('CIK:')
        CompanyName = CompNameCIK[0]
        CIK = CompNameCIK[1]
#     else:
#         ementa = None




    if 'operation' in statements_data[each]['headers'][0][0].lower() or 'income' in statements_data[each]['headers'][0][0].lower() or 'earning' in statements_data[each]['headers'][0][0].lower():
#         or 'cash flow' in statements_data[each]['headers'][0][0].lower():
#     or 'consolidated' in statements_data[each]['headers'][0][0].lower() or 'statement' in statements_data[each]['headers'][0][0].lower() or 'sheet' in statements_data[each]['headers'][0][0].lower():



####################################################################################################################   



        NetRevenue = pd.DataFrame(statements_data[each]['data']).loc[pd.DataFrame(statements_data[each]['data']).iloc[:,0].apply(lambda x: x.lower()).str.contains('|'.join(['revenue','revenues',
        'sale','sales','net loss','net (loss)',
        'net interest income','total operating','interest income','net interest',
#         'net cash','earning','net other',
#         'asset','liabilit',
#         'general and administrative','general & administrative',#selling, 
        'research and development','research & development', 'R&D','R & D',
#         'effective tax rate','effective rate'
])),:]

#######Using "in" instead of str.contains:
#         stat_df=pd.DataFrame(statements_data[each]['data'])
#         NetRevenue = stat_df[stat_df[0].apply(lambda x: any(term in x.lower() for term in ['revenues', 'revenue','sale','sales',
#                                                                                  'net loss','net (loss)','net interest income',
#                                                                                  'total operating','interest income','net interest',
#                                                                                            'research and development','research & development', 'R&D','R & D']))]

#         df1=NetRevenue.replace("\$",'',regex=True)
        df = NetRevenue[NetRevenue.columns].replace({'\(':'-','\)':'','â€‹':'','%':'','\$':'',' ':'',',':'','[1]':'','[2]':''}, regex = True)
    # using apply method
        cols = df.columns.drop(df.columns[0])
        df[cols]=df[cols].apply(lambda x: pd.to_numeric(x, errors='coerce'))####Converting non metric columns to numeric to be able to multiply later with


        if len(df)!=0:
            df_Revenue=df

            if '$ in millions' in statements_data[each]['headers'][0][0].lower():# and '$' in NetRevenue:
               #and NetRevenue>1000  
            ##Cleaning the data before building it:
                s =df_Revenue[df_Revenue.select_dtypes(include=['number']).columns]*1000000
                df[s.columns] = s
            elif '$ in thousands' in statements_data[each]['headers'][0][0].lower():# and '$' in NetRevenue:
                s =df_Revenue[df_Revenue.select_dtypes(include=['number']).columns]*1000
                df[s.columns] = s
            elif '$ in billions' in statements_data[each]['headers'][0][0].lower():# and '$' in NetRevenue:
                s =df_Revenue[df_Revenue.select_dtypes(include=['number']).columns]*1000000000
                df[s.columns] = s
                ## below is an exception
    #         elif '$ in millions, $ in billions' in statements_data[each]['headers'][0][0].lower():# and '$' in NetRevenue:
    #             # and NetRevenue<1000 
    #             df_Revenue =df_Revenue[df_Revenue.select_dtypes(include=['number']).columns]*1000000000
            else:
                s=df_Revenue[df_Revenue.select_dtypes(include=['number']).columns]*1
                df[s.columns] = s

            df1B_duped = pd.concat([df[df.columns[0]], df_Revenue], axis=1)


            ####Dedupe column names right here to change colnames ############


            df1B_dedupe = df1B_duped.loc[:,~df1B_duped.columns.duplicated()].copy()

            ##for later
            df1B=df1B_dedupe.dropna(axis=1)
    ## Below is to avoid reindex error due to repeated columns like deferred tax revenue being reported in two columns acc to: https://stackoverflow.com/questions/14984119/python-pandas-remove-duplicate-columns
    #         df1B = pd.concat([df[df.columns[0]], df_Revenue], axis=1).loc[:,~(pd.concat([df[df.columns[0]], df_Revenue], axis=1)).columns.duplicated()].copy()

    #             df1B = df_Rev[df_Rev.columns].replace({'\(':'-','\)':'','â€‹':'','%':'','\$':'',' ':'',',':''}, regex = True)
        #         '\$':''<<-removed this condition because if there is $ in front of the value, the headers direct the units thousand or million

                #         if '$'in df1B:
                    #multiply bymillion or thousand
            df_out1B=pd.DataFrame()
            ##2. Replace NanNs with next column's non-NaN value
            if df1B.isnull().values.any():
                df_out1B = df1B.fillna(method='ffill',axis=0).dropna(axis=1)
    #                 df_out1B = df1B.apply(lambda x: pd.Series(x.dropna().to_numpy()), axis=1)
    #                 df_out1B = df_out1B.set_axis(df1B.columns[:df_out1B.shape[1]], axis=1).reindex(df1B.columns, axis=1).dropna(axis=1)
            else:
                df_out1B = df1B.dropna(axis=1)
            print(df_out1B)

            ##3. replacing empty whitespace with NaN---do we need to do this again?
            cleaned_NetRevenue=df_out1B.replace(r'^\s*$', np.nan, regex=True).dropna(axis=1, how='all')


            if len(CompNameCIK)!=0:
                cleaned_NetRevenue['Company']= CompanyName
                cleaned_NetRevenue['cik_str']= CIK
            else:
                cleaned_NetRevenue['Company']= 'NotFound'
                cleaned_NetRevenue['cik_str']= 'NotFound'
            print(cleaned_NetRevenue) 

    #######################To avoid the initial columns being 0,1,2,3 followed with opsheader dates##############


    ##################################################################################
        ##To avoid clumping all dates into a single list:
            if len(statements_data[each]['headers'])>1:
                        dates_init = statements_data[each]['headers'][1]
            else:
                dates_init = statements_data[each]['headers'][0]


            dateslist_initial=[]
            for onedate in dates_init:
            #     print(re.sub(r'(,?!(([^"]*"){2})*[^"]*$)',':', str(onedate)))
                mystring1 = re.sub(r'(,?!(([^"]*"){2})*[^"]*$)',':', str(onedate))
                print(re.sub(",", "/", mystring1))
                datestr1a=re.sub(",", "/", mystring1)## replace comma with / before the year
                if len(datestr1a.split("/"))>1:
                    datestr1=datestr1a.split("/")[0].split(" ")[0]+" "+datestr1a.split("/")[1]
                    ##To drop the exact date, splitting at \ for year and then splitting at space to keep Month
                else:
                    datestr1=datestr1a.split("/")[0].split(" ")[0]+" "+datestr1a.split("/")[0]
                dateslist_initial.append(datestr1)

            print(dateslist_initial)
            singledatesstring1 = ','.join(dateslist_initial)
            datesList1_init = re.sub(",", ",", singledatesstring1).split(',')

            CleanRevHeaders = ['Metric']


            for each1 in datesList1_init:
                print('-'.join(each1.split(',')))
                CleanRevHeaders.append(''.join(each1.split(',')))


            import itertools
            AllCleanRevHeaders = list(itertools.chain(CleanRevHeaders, ['Company'], ['cik_str']))

            if len(cleaned_NetRevenue.columns.tolist())==len(AllCleanRevHeaders):
                cleaned_NetRevenue.columns=AllCleanRevHeaders

            print(cleaned_NetRevenue)
        ##################################################################################

    ####################################################################################################################


    ##Some tables might not be there like R&D for some companies:
            if (len(cleaned_NetRevenue))!=0:#+len(cleaned_NetSales)+len(cleaned_OperatingCost)+len(cleaned_EmployeeWages)+len(cleaned_RandDOpsAmgen))!=0:
                #ops = concat(*(cleaned_NetRevenue))#, cleaned_NetSales,cleaned_OperatingCost, cleaned_EmployeeWages,cleaned_RandDOpsAmgen))
                ops=cleaned_NetRevenue
                if len(ops)!=0:

                    editedOps = pd.DataFrame(ops[ops.columns[ops.notnull().all()]].iloc[0].dropna())
                    editedOps.dropna(inplace=True)
                    editedOps
                    # remove empty column that just takes up space and shifts the 3 years' data one cell further
                    CleanOps = editedOps.loc[:, editedOps.notna().any()]
                    Operations_edit = pd.DataFrame(CleanOps.replace(r'^\s*$%', np.nan, regex=True).dropna())
                    #TotalUStax_edit.where(~TotalUStax_edit.apply(lambda x: x.astype(str).str.contains('|'.join(['%','$'])))).dropna().T




        #             Ops_sel = [ops]
                    Ops_sel = Operations_edit.T
        #             dates = statements_data[each]['headers'][0]
                    if len(statements_data[each]['headers'])>1:
                        dates = statements_data[each]['headers'][1]
                    else:
                        dates = statements_data[each]['headers'][0]

    #                     Ops_sel.columns = ['Metric',dates,'Company','cik_str']##<<<<to see if date can be registered


        ##################################################################################
        ##To avoid clumping all dates into a single list:

                    dateslist1=[]
                    for onedate in dates:
                    #     print(re.sub(r'(,?!(([^"]*"){2})*[^"]*$)',':', str(onedate)))
                        mystring1 = re.sub(r'(,?!(([^"]*"){2})*[^"]*$)',':', str(onedate))
                        print(re.sub(",", "/", mystring1))
                        datestr1a=re.sub(",", "/", mystring1)## replace comma with / before the year
                        if len(datestr1a.split("/"))>1:
                            datestr1=datestr1a.split("/")[0].split(" ")[0]+" "+datestr1a.split("/")[1]
                            ##To drop the exact date, splitting at \ for year and then splitting at space to keep Month
                        else:
                            datestr1=datestr1a.split("/")[0].split(" ")[0]+" "+datestr1a.split("/")[0]
                        dateslist1.append(datestr1)

                    print(dateslist1)
                    singledatesstring1 = ','.join(dateslist1)
                    datesList1 = re.sub(",", ",", singledatesstring1).split(',')

                    OpsHeaders = ['Metric']


                    for each1 in datesList1:
                        print('-'.join(each1.split(',')))
                        OpsHeaders.append(''.join(each1.split(',')))


                    import itertools
                    AllOpsHeaders = list(itertools.chain(OpsHeaders, ['Company'], ['cik_str']))

                    if len(Ops_sel.columns.tolist())==len(AllOpsHeaders):
                        Ops_sel.columns=AllOpsHeaders
        ##################################################################################
        #         else:
        #             Ops_sel = pd.DataFrame(['NIL'])

        ##### Checking for duplicated columns like deferred revenues that create problems in rbind:

            Ops_sel_dedupe = Ops_sel.loc[:,~Ops_sel.columns.duplicated()].copy()
            OperationsStatement.append(Ops_sel_dedupe)
            print(OperationsStatement)


            ##Also creating a dataframe instead of a list of dataframes for easier understanding of column values
            DFOperationsStatement2 = pd.concat([DFOperationsStatement2, Ops_sel_dedupe])

            # Define the filename for the current batch
    #             filename = f'batch_{each+1}_{Outereach+1}.csv'
    #             print(filename)
    #             batch_df=DFOperationsStatement
            # Export the current batch to a CSV file
    #             batch_df.to_csv(filename, index=False)
    #             batch_df.to_csv(DFOperationsStatement, index=False)
else:
    print("*"*100)
    print('None of the listed tables')
    print("*"*100)



    CompanyFinancials = [OperationsStatement]#,TotalUStax,RandD]
AllTablesOfACompany.append(CompanyFinancials)
    

# OuterDFOperationsStatement= pd.concat([OuterDFOperationsStatement, DFOperationsStatement])


end_AllTables = time.time()

timeTaken = end_AllTables-start_AllTables

DFOperationsStatement2.to_csv('C:/UCLAAnderson/10KfilingsTables/Data/AutomatedFinancialData/May19B_Part2_2023_withMetricsnDate2021nUnits_RevenueRnD_SP500.csv', index=False)


In [ ]:
DFOperationsStatement.to_csv('C:/UCLAAnderson/10KfilingsTables/Data/AutomatedFinancialData/May17B_2023_withMetricsnDate2021nUnits_RevenueRnD_SP500.csv', index=False)
print(timeTaken/60)


In [ ]:
len(statements_data)
list(range(((round(len(statements_data)/2))+1),len(statements_data)))
list(range(0,round(len(statements_data)/2),1))
round(len(statements_data)/2)

In [ ]:
# print(timeTaken/60)
cleaned_NetRevenue
df_Revenue
NetRevenue
statements_data[each]
DFOperationsStatement.tail(25)

In [ ]:
statements_data[each]
# NetRevenue
# pd.DataFrame(statements_data[each]['data']).loc[pd.DataFrame(statements_data[each]['data']).iloc[:,0].apply(lambda x: x.lower()).str.contains('|'.join(['revenue','revenues',
#     'sale','sales','net loss','net (loss)',
#     'net interest income','total operating','interest income','net interest','research and development','research & development', 'R&D','R & D'
# #         'net cash','earning','net other',
# #         'asset','liabilit',
# #         'general and administrative','general & administrative',#selling, 
    
# #         'effective tax rate','effective rate'
# ])),:]
df1B_dedupe
cleaned_NetRevenue
Ops_sel

len(NetRevenue)

### Exporting the output #####

In [ ]:
DFOperationsStatement=pd.concat(DFOperationsStatement1,DFOperationsStatement2)
DFOperationsStatement.to_csv('C:/UCLAAnderson/10KfilingsTables/Data/AutomatedFinancialData/May19_2023_withMetricsnDate2021nUnits_RevenueRnD_SP563.csv', index=False)

# OuterDFOperationsStatement.to_csv('C:/UCLAAnderson/10KfilingsTables/Data/AutomatedFinancialData/May11B_2023_AllwithMetricsnDate2021nUnits_RevenueRnD_SP563.csv', index=False)


In [ ]:
AllTablesOfACompany

## Debugged

## Parallel Processing

In [ ]:
# import concurrent.futures
# from functools import partial

In [ ]:
# def process_statement_data(statement_data):
# #     for each in list(range(0,len(statements_data),1)):#batch:
#     income_headers = statements_data['headers']##for taking the dates stored as the second list in headers of 
#                                                      ##Consolidated Statements of Operations and Comprehensive Income (Loss) - USD ($) $ in Thousands

#         ##The balance sheet has only one header but if you go to the income statement it has two headers 
#         ##we take [1] because we were interested in the dates header which was the second element

#     income_data = statements_data['data']
#     # income_data ##here we have each row and the corresponding value for each particular financial statement- different sections


#     ## Put the data in a dataframe
#     income_df = pd.DataFrame(income_data)

#     ## So, define the index column, rename it, and we need to make sure to drop the old column once we reindex.

#     income_df.index = income_df[0]##column 0 is first column-- something like row.names in R

#     ## Sometimes before removing the first column to row.names instead of replacing you can also assign it a new intuitive name

#     income_df.index.name = 'Category'
#     ##now dropping the first column which is now a rowname or index:
#     income_df = income_df.drop(0, axis = 1) ##drop method takes the column number to be dropped and along which axis 
#                                             ##- 1 means along column axis

#     ## Now, we see problem 2, which is the dollar signs -  which tells me that right now it is a string
#     ## this being a string is a problem because I can't do any calculations on a string
#     ## Additionally, the blank values should be represented as a non-value (Problem 3)
#     ## 4th problem: negative values in losses should be true negatives and not have $

#     ## So, running some replace operations on this dataframe

#     ## Get rid of the '$', '(', ')', and convert the '' to NaNs. @44:34
#     #dealing with replacing positive values
#     #then, dealing with negative values such that if you see closing bracket replace with minus sign
#     # finally, if you see empty cells, replace with NaNs non values
#     income_df = income_df.replace('[\$,)]','', regex = True)\
#     .replace('[(]','-', regex = True)\
#     .replace('',np.nan, regex = True)


#     CompNameCIK=str()
#     NetRevenue=[]
#     NetSales=[]
#     OperatingCost=[]
#     EmployeeWages=[]
# #         OperatingCost=pd.DataFrame()
# #         Ops_sel=pd.DataFrame()
# #         ops=pd.DataFrame()
#     ##Now that we aren't relying on the cover page to get the name of 

# #     soupy=BeautifulSoup(requests.get(statements_urlDF['LinkToAllTabs'][0], headers = headers).content,'html.parser')
#     # Iterate over the list of URLs
#     urlDF=pd.concat(statements_urlDF)
#     for url in urlDF:
#         soupy = BeautifulSoup(requests.get(url, headers=headers).content, 'html.parser')
#         node = soupy.find('span', class_='companyName')
#         if node is not None:
#             ementa = node.text
#             CompNameCIK=soupy.find('span', class_='companyName').text.strip().split('CIK:')
#             CompanyName = CompNameCIK[0]
#             CIK = CompNameCIK[1]
# #     else:
# #         ementa = None




#     if 'operation' in statements_data['headers'][0][0].lower() or 'income' in statements_data['headers'][0][0].lower():
# #         or 'cash flow' in statements_data[each]['headers'][0][0].lower():
# #     or 'consolidated' in statements_data[each]['headers'][0][0].lower() or 'statement' in statements_data[each]['headers'][0][0].lower() or 'sheet' in statements_data[each]['headers'][0][0].lower():



# ####################################################################################################################   



#         NetRevenue = pd.DataFrame(statements_data['data']).loc[pd.DataFrame(statements_data['data']).iloc[:,0].apply(lambda x: x.lower()).str.contains('|'.join(['revenue',
#         'net income','sale','net loss','net (loss)','total sales',
#         'net interest income','total operating','interest income','net interest',
# #         'net cash','earning','net other',
# #         'asset','liabilit',
# #         'general and administrative','general & administrative',#selling, 
#         'research and development','research & development', 'R&D','R & D',
# #         'effective tax rate','effective rate'
# ])),:]


# #         df1=NetRevenue.replace("\$",'',regex=True)
#         df = NetRevenue[NetRevenue.columns].replace({'\(':'-','\)':'','â€‹':'','%':'','\$':'',' ':'',',':'','[1]':'','[2]':''}, regex = True)
#     # using apply method
#         cols = df.columns.drop(df.columns[0])
#         df[cols]=df[cols].apply(lambda x: pd.to_numeric(x, errors='coerce'))####Converting non metric columns to numeric to be able to multiply later with
#         df_Revenue=df

#         if '$ in millions' in statements_data['headers'][0][0].lower():# and '$' in NetRevenue:
#            #and NetRevenue>1000  
#         ##Cleaning the data before building it:
#             s =df_Revenue[df_Revenue.select_dtypes(include=['number']).columns]*1000000
#             df[s.columns] = s
#         elif '$ in thousands' in statements_data['headers'][0][0].lower():# and '$' in NetRevenue:
#             s =df_Revenue[df_Revenue.select_dtypes(include=['number']).columns]*1000
#             df[s.columns] = s
#         elif '$ in billions' in statements_data['headers'][0][0].lower():# and '$' in NetRevenue:
#             s =df_Revenue[df_Revenue.select_dtypes(include=['number']).columns]*1000000000
#             df[s.columns] = s
#             ## below is an exception
# #         elif '$ in millions, $ in billions' in statements_data[each]['headers'][0][0].lower():# and '$' in NetRevenue:
# #             # and NetRevenue<1000 
# #             df_Revenue =df_Revenue[df_Revenue.select_dtypes(include=['number']).columns]*1000000000
#         else:
#             s=df_Revenue[df_Revenue.select_dtypes(include=['number']).columns]*1
#             df[s.columns] = s
# #             df[df.select_dtypes(include=['number']).columns] *= 3

# #         print(df_Revenue)

#             ## After this, we can remove the $ sign
#             ##1. Remove any/all % and $ signs that take up separate columns or within a column preceding the value because we are only studying US$
# #             df1B=df_Revenue[df_Revenue.columns].replace({'\(':'-','\)':'','â€‹':'','%':'','\$':''}, regex = True)## need to precede the dollar sign with a \ to be identified in regex

#             ###Concatenate cbind the net sales/ revenue/ R&D terminologies with multiplied values:
# #         df1B = df_Revenue
# #         df = df.reset_index(drop=True)
#         df1B_duped = pd.concat([df[df.columns[0]], df_Revenue], axis=1)


#         ####Dedupe column names right here to change colnames ############


#         df1B_dedupe = df1B_duped.loc[:,~df1B_duped.columns.duplicated()].copy()

#         ##for later
#         df1B=df1B_dedupe.dropna()
# ## Below is to avoid reindex error due to repeated columns like deferred tax revenue being reported in two columns acc to: https://stackoverflow.com/questions/14984119/python-pandas-remove-duplicate-columns
# #         df1B = pd.concat([df[df.columns[0]], df_Revenue], axis=1).loc[:,~(pd.concat([df[df.columns[0]], df_Revenue], axis=1)).columns.duplicated()].copy()

# #             df1B = df_Rev[df_Rev.columns].replace({'\(':'-','\)':'','â€‹':'','%':'','\$':'',' ':'',',':''}, regex = True)
#     #         '\$':''<<-removed this condition because if there is $ in front of the value, the headers direct the units thousand or million

#             #         if '$'in df1B:
#                 #multiply bymillion or thousand
#         df_out1B=pd.DataFrame()
#         ##2. Replace NanNs with next column's non-NaN value
#         if df1B.isnull().values.any():
#             df_out1B = df1B.fillna(method='ffill',axis=0).dropna(axis=1)
# #                 df_out1B = df1B.apply(lambda x: pd.Series(x.dropna().to_numpy()), axis=1)
# #                 df_out1B = df_out1B.set_axis(df1B.columns[:df_out1B.shape[1]], axis=1).reindex(df1B.columns, axis=1).dropna(axis=1)
#         else:
#             df_out1B = df1B.dropna(axis=1)
#         print(df_out1B)

#         ##3. replacing empty whitespace with NaN---do we need to do this again?
#         cleaned_NetRevenue=df_out1B.replace(r'^\s*$', np.nan, regex=True).dropna(axis=1, how='all')


#         if len(CompNameCIK)!=0:
#             cleaned_NetRevenue['Company']= CompanyName
#             cleaned_NetRevenue['cik_str']= CIK
#         else:
#             cleaned_NetRevenue['Company']= 'NotFound'
#             cleaned_NetRevenue['cik_str']= 'NotFound'
#         print(cleaned_NetRevenue) 

# #######################To avoid the initial columns being 0,1,2,3 followed with opsheader dates##############


# ##################################################################################
#     ##To avoid clumping all dates into a single list:
#         if len(statements_data['headers'])>1:
#                     dates_init = statements_data['headers'][1]
#         else:
#             dates_init = statements_data['headers'][0]


#         dateslist_initial=[]
#         for onedate in dates_init:
#         #     print(re.sub(r'(,?!(([^"]*"){2})*[^"]*$)',':', str(onedate)))
#             mystring1 = re.sub(r'(,?!(([^"]*"){2})*[^"]*$)',':', str(onedate))
#             print(re.sub(",", "/", mystring1))
#             datestr1a=re.sub(",", "/", mystring1)## replace comma with / before the year
#             if len(datestr1a.split("/"))>1:
#                 datestr1=datestr1a.split("/")[0].split(" ")[0]+" "+datestr1a.split("/")[1]
#                 ##To drop the exact date, splitting at \ for year and then splitting at space to keep Month
#             else:
#                 datestr1=datestr1a.split("/")[0].split(" ")[0]+" "+datestr1a.split("/")[0]
#             dateslist_initial.append(datestr1)

#         print(dateslist_initial)
#         singledatesstring1 = ','.join(dateslist_initial)
#         datesList1_init = re.sub(",", ",", singledatesstring1).split(',')

#         CleanRevHeaders = ['Metric']


#         for each1 in datesList1_init:
#             print('-'.join(each1.split(',')))
#             CleanRevHeaders.append(''.join(each1.split(',')))


#         import itertools
#         AllCleanRevHeaders = list(itertools.chain(CleanRevHeaders, ['Company'], ['cik_str']))

#         if len(cleaned_NetRevenue.columns.tolist())==len(AllCleanRevHeaders):
#             cleaned_NetRevenue.columns=AllCleanRevHeaders

        
#     ##################################################################################

# ####################################################################################################################


# ##Some tables might not be there like R&D for some companies:
#         if (len(cleaned_NetRevenue))!=0:#+len(cleaned_NetSales)+len(cleaned_OperatingCost)+len(cleaned_EmployeeWages)+len(cleaned_RandDOpsAmgen))!=0:
#             #ops = concat(*(cleaned_NetRevenue))#, cleaned_NetSales,cleaned_OperatingCost, cleaned_EmployeeWages,cleaned_RandDOpsAmgen))
#             ops=cleaned_NetRevenue
#             if len(ops)!=0:

#                 editedOps = pd.DataFrame(ops[ops.columns[ops.notnull().all()]].iloc[0].dropna())
#                 editedOps.dropna(inplace=True)
#                 editedOps
#                 # remove empty column that just takes up space and shifts the 3 years' data one cell further
#                 CleanOps = editedOps.loc[:, editedOps.notna().any()]
#                 Operations_edit = pd.DataFrame(CleanOps.replace(r'^\s*$%', np.nan, regex=True).dropna())
#                 #TotalUStax_edit.where(~TotalUStax_edit.apply(lambda x: x.astype(str).str.contains('|'.join(['%','$'])))).dropna().T




#     #             Ops_sel = [ops]
#                 Ops_sel = Operations_edit.T
#     #             dates = statements_data[each]['headers'][0]
#                 if len(statements_data['headers'])>1:
#                     dates = statements_data['headers'][1]
#                 else:
#                     dates = statements_data['headers'][0]

# #                     Ops_sel.columns = ['Metric',dates,'Company','cik_str']##<<<<to see if date can be registered


#     ##################################################################################
#     ##To avoid clumping all dates into a single list:

#                 dateslist1=[]
#                 for onedate in dates:
#                 #     print(re.sub(r'(,?!(([^"]*"){2})*[^"]*$)',':', str(onedate)))
#                     mystring1 = re.sub(r'(,?!(([^"]*"){2})*[^"]*$)',':', str(onedate))
#                     print(re.sub(",", "/", mystring1))
#                     datestr1a=re.sub(",", "/", mystring1)## replace comma with / before the year
#                     if len(datestr1a.split("/"))>1:
#                         datestr1=datestr1a.split("/")[0].split(" ")[0]+" "+datestr1a.split("/")[1]
#                         ##To drop the exact date, splitting at \ for year and then splitting at space to keep Month
#                     else:
#                         datestr1=datestr1a.split("/")[0].split(" ")[0]+" "+datestr1a.split("/")[0]
#                     dateslist1.append(datestr1)

#                 print(dateslist1)
#                 singledatesstring1 = ','.join(dateslist1)
#                 datesList1 = re.sub(",", ",", singledatesstring1).split(',')

#                 OpsHeaders = ['Metric']


#                 for each1 in datesList1:
#                     print('-'.join(each1.split(',')))
#                     OpsHeaders.append(''.join(each1.split(',')))


#                 import itertools
#                 AllOpsHeaders = list(itertools.chain(OpsHeaders, ['Company'], ['cik_str']))

#                 if len(Ops_sel.columns.tolist())==len(AllOpsHeaders):
#                     Ops_sel.columns=AllOpsHeaders
#     ##################################################################################
#     #         else:
#     #             Ops_sel = pd.DataFrame(['NIL'])

#     ##### Checking for duplicated columns like deferred revenues that create problems in rbind:

#         Ops_sel_dedupe = Ops_sel.loc[:,~Ops_sel.columns.duplicated()].copy()
#         OperationsStatement.append(Ops_sel_dedupe)
#         print(OperationsStatement)


#         ##Also creating a dataframe instead of a list of dataframes for easier understanding of column values
#         DFOperationsStatement = pd.concat([DFOperationsStatement, Ops_sel_dedupe])

#         # Define the filename for the current batch
# #             filename = f'batch_{each+1}_{Outereach+1}.csv'
# #             print(filename)
# #             batch_df=DFOperationsStatement
#         # Export the current batch to a CSV file
# #             batch_df.to_csv(filename, index=False)
# #             batch_df.to_csv(DFOperationsStatement, index=False)
#     return(DFOperationsStatement)
 

In [ ]:
# # Create a list of statement data
# statement_data_list = statements_data

# # Define the number of concurrent workers (adjust as needed)
# num_workers = 4

# # Create a ThreadPoolExecutor with the desired number of workers
# executor = concurrent.futures.ThreadPoolExecutor(max_workers=num_workers)

# # Submit the tasks to the executor
# # Use partial to create a new function with a single argument for statement_data
# # Pass the statement_data_list to map for parallel processing
# future_results = executor.map(partial(process_statement_data), statement_data_list)

# # Retrieve the results from the futures
# results = list(future_results)

# # Shutdown the executor to free resources
# executor.shutdown()


##### --------------- end parallel processing -----------

In [ ]:
##################################################################################

# END

************************************************************************